<a href="https://colab.research.google.com/github/JARIN-TIAS/NLP/blob/main/Emotion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from transformers import pipeline

# Load pre-trained emotion detection model
emotion_classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base")

# Function for emotion detection
def detect_emotion(text):
    result = emotion_classifier(text)
    return result

# Sample sentences for testing
sample_texts = [
    "I am so happy today!",
    "I feel really sad and lonely.",
    "I am angry because of the traffic.",
    "This is so exciting, I can't wait!",
    "I am worried about the future.",
]

# Detect emotions for each sentence
for text in sample_texts:
    print(f"Text: {text}")
    emotion = detect_emotion(text)
    print(f"Detected Emotion: {emotion[0]['label']} (Confidence: {emotion[0]['score']:.2f})\n")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Text: I am so happy today!
Detected Emotion: joy (Confidence: 0.96)

Text: I feel really sad and lonely.
Detected Emotion: sadness (Confidence: 0.99)

Text: I am angry because of the traffic.
Detected Emotion: anger (Confidence: 0.99)

Text: This is so exciting, I can't wait!
Detected Emotion: joy (Confidence: 0.94)

Text: I am worried about the future.
Detected Emotion: fear (Confidence: 0.99)



# **for a post in tweeter **

In [ ]:
import tweepy
from transformers import pipeline
import pandas as pd
import re

# Set up Twitter API credentials (fill in your own credentials)
consumer_key = 'YOUR_CONSUMER_KEY'
consumer_secret = 'YOUR_CONSUMER_SECRET'
access_token = 'YOUR_ACCESS_TOKEN'
access_token_secret = 'YOUR_ACCESS_TOKEN_SECRET'

# Set up Tweepy authentication
auth = tweepy.OAuth1UserHandler(
    consumer_key, consumer_secret, access_token, access_token_secret
)
api = tweepy.API(auth)

# Load pre-trained emotion detection model
emotion_classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base")

# Function to fetch tweets about a specific topic
def fetch_tweets(query, count=100):
    tweets = tweepy.Cursor(api.search_tweets, q=query, lang="en", tweet_mode="extended").items(count)
    tweet_data = []

    for tweet in tweets:
        tweet_data.append({
            "tweet": tweet.full_text,
            "user": tweet.user.screen_name,
            "date": tweet.created_at
        })

    return pd.DataFrame(tweet_data)

# Function to clean tweet text
def clean_tweet(tweet):
    # Remove URLs, mentions, hashtags, and special characters
    tweet = re.sub(r'http\S+|www\S+|https\S+', '', tweet)  # Remove URLs
    tweet = re.sub(r'@\w+', '', tweet)  # Remove mentions (@username)
    tweet = re.sub(r'#\w+', '', tweet)  # Remove hashtags
    tweet = re.sub(r'[^\w\s]', '', tweet)  # Remove punctuation
    tweet = tweet.strip()
    return tweet

# Function to detect emotion from text
def detect_emotion(text):
    result = emotion_classifier(text)
    return result[0]['label'], result[0]['score']

# Fetch tweets on a specific topic (e.g., "climate change")
topic = "climate change"
tweets_df = fetch_tweets(topic, count=100)  # Get the last 100 tweets

# Clean tweets and detect emotions
tweets_df['cleaned_tweet'] = tweets_df['tweet'].apply(clean_tweet)
tweets_df['emotion'], tweets_df['emotion_score'] = zip(*tweets_df['cleaned_tweet'].apply(detect_emotion))

# Show the resulting dataframe
print(tweets_df[['user', 'date', 'cleaned_tweet', 'emotion', 'emotion_score']])

# Optionally, save to a CSV file
tweets_df.to_csv('tweets_emotion_detection.csv', index=False)
